In [0]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

In [9]:
train=pd.read_csv('data/emnist-balanced-train.csv',header=None)
test=pd.read_csv('data/emnist-balanced-test.csv',header=None)
print('trainData : ')
train.head()

trainData : 


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,36,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,43,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
train_data=train.iloc[:100000,1:]
train_label=train.iloc[:100000,0]
vaild_data=train.iloc[100000:,1:]
vaild_label=train.iloc[100000:,0]
test_data=test.iloc[:,1:]
test_label=test.iloc[:,0]

In [11]:
train_label = pd.get_dummies(train_label)
test_label = pd.get_dummies(test_label)
vaild_label=pd.get_dummies(vaild_label)
train_label.head()

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
train_data=train_data.values
train_label=train_label.values
vaild_data=vaild_data.values
vaild_label=vaild_label.values
test_data=test_data.values
test_label=test_label.values




In [0]:
def rotate(image):
    image = image.reshape([28, 28])
    image = np.fliplr(image)
    image = np.rot90(image)
    
    
    return image.reshape([28 * 28])
train_data = np.apply_along_axis(rotate, 1, train_data)/255
vaild_data=np.apply_along_axis(rotate,1,vaild_data)/255
test_data = np.apply_along_axis(rotate, 1, test_data)/255


In [14]:
test_data=test_data.reshape([test_data.shape[0],28,28,1])
train_data=train_data.reshape([train_data.shape[0],28,28,1])
vaild_data=vaild_data.reshape([vaild_data.shape[0],28,28,1])
print(test_data.shape)
print(train_data.shape)
print(vaild_data.shape)
print(len(test_data))

(18800, 28, 28, 1)
(100000, 28, 28, 1)
(12800, 28, 28, 1)
18800


In [0]:
def processing_img(img):
    img=img.reshape([28 , 28])
    img=np.round(img)
    img=img.reshape([28,28,1])
    return img

def processing_imgs(imgs):
    list_of_imgs=[processing_img(img) for img in imgs]
    return list_of_imgs
    

trainP_data = np.asarray(processing_imgs(train_data))
vaildP_data=np.asarray(processing_imgs(vaild_data))
testP_data = np.asarray(processing_imgs(test_data))


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [17]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', 
                        input_shape=(28, 28, 1)))

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"))
model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"))
model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(47, activation='softmax')) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
!mkdir /content/saved_models

In [22]:
from keras.callbacks import ModelCheckpoint 
epochs = 5


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.CNN.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(trainP_data, train_label, validation_data=(vaildP_data, vaild_label), epochs=epochs,callbacks=[checkpointer],
          batch_size=100, verbose=1)

Train on 100000 samples, validate on 12800 samples
Epoch 1/5
100000/100000 [==============================] - 499s 5ms/step - loss: 0.3673 - acc: 0.8665 - val_loss: 0.3259 - val_acc: 0.8787

Epoch 00001: val_loss improved from inf to 0.32587, saving model to saved_models/weights.best.CNN.hdf5
Epoch 2/5
100000/100000 [==============================] - 493s 5ms/step - loss: 0.3560 - acc: 0.8692 - val_loss: 0.3238 - val_acc: 0.8770

Epoch 00002: val_loss improved from 0.32587 to 0.32382, saving model to saved_models/weights.best.CNN.hdf5
Epoch 3/5
100000/100000 [==============================] - 495s 5ms/step - loss: 0.3433 - acc: 0.8740 - val_loss: 0.3098 - val_acc: 0.8812

Epoch 00003: val_loss improved from 0.32382 to 0.30981, saving model to saved_models/weights.best.CNN.hdf5
Epoch 4/5
100000/100000 [==============================] - 498s 5ms/step - loss: 0.3358 - acc: 0.8766 - val_loss: 0.3083 - val_acc: 0.8827

Epoch 00004: val_loss improved from 0.30981 to 0.30832, saving model to 

In [0]:
model.load_weights('saved_models/weights.best.CNN.hdf5')



In [26]:
score = model.evaluate(testP_data, test_label, verbose=0)
accuracy = 100*score[1]
# accuracy_values['CNN'] = accuracy
# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

Test accuracy: 88.8777%
